In [1]:
!pip3 install transformers>=4.32.0 optimum>=1.12.0
!pip3 install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/  # Use cu117 if on CUDA 11.7


Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu118/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 28.0 MB/s eta 0:00:00


In [2]:
!pip install transformers accelerate optimum -q

In [3]:
!pip install transformers>=4.33.0 optimum>=1.12.0

In [4]:
!pip install optimum

In [5]:
pip install auto-gptq

In [6]:
import torch
print(torch.__version__)

2.2.1+cu121


In [7]:
# pip install accelerate

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name_or_path = "TheBloke/Llama-2-13B-chat-GPTQ"
# To use a different branch, change revision
# For example: revision="main"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="cuda",
                                             torch_dtype=torch.float16,
                                             trust_remote_code=True)

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/837 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/7.26G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4193: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
The cos_cached attribute will be removed in 4.39. Bear in mind that its contents changed in v4.38. Use the forward method of RoPE from now on instead. It is not used in the `LlamaAttention` class
The sin_cached attribute will be removed in 4.39. Bear in mind that its contents changed in v4.38. Use the forward method of RoPE from now on instead. It is not used in the `LlamaAttention` class


generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [10]:
import json

# Load JSON file
with open('/content/drive/MyDrive/datasets/commonsense_qa.json', 'r') as file:
    data = json.load(file)

# Initialize an empty list to store results


In [11]:
results = []
count = 0
for item in data['data']:
  count= count+1
  if count > 900:
    prompt = item['question']

    print(count)
    # prompt_template=f'''[INST] <<SYS>>
    # You are an online math tutor bot. Follow the given instructions: Given the below-structured query,
    # your job is to understand the word problem mentioned as "question" and explain the solution in numbered
    # steps to get the correct "answer". Begin your response with "let's think step by step" and append Answer: ["answer" from the
    # below-structured query] at the end of your reasoning. Strictly limit the reasoning to 20-30 words. Do not use emojis.{prompt}[/INST]
    # '''
    prompt_template=f'''[INST] <<SYS>>Lets think step by step. {prompt}[/INST]
    '''
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.1,
        top_p=0.95,
        top_k=1,
        repetition_penalty=1.1
    )
    response = pipe(prompt_template)[0]['generated_text']
    response = response.split('[/INST]')[1].strip()

    results.append({'prompt': item['question'], 'reasoning': response, 'answer': item['answer']})
    if count==1200:
      break


901


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1339: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


902
903
904
905
906
907
908
909
910
911
912
913
914
915
916
917
918
919
920
921
922
923
924
925
926
927
928
929
930
931
932
933
934
935
936
937
938
939
940
941
942
943
944
945
946
947
948
949
950
951
952
953
954
955
956
957
958
959
960
961
962
963
964
965
966
967
968
969
970
971
972
973
974
975
976
977
978
979
980
981
982
983
984
985
986
987
988
989
990
991
992
993
994
995
996
997
998
999
1000
1001
1002
1003
1004
1005
1006
1007
1008
1009
1010
1011
1012
1013
1014
1015
1016
1017
1018
1019
1020
1021
1022
1023
1024
1025
1026
1027
1028
1029
1030
1031
1032
1033
1034
1035
1036
1037
1038
1039
1040
1041
1042
1043
1044
1045
1046
1047
1048
1049
1050
1051
1052
1053
1054
1055
1056
1057
1058
1059
1060
1061
1062
1063
1064
1065
1066
1067
1068
1069
1070
1071
1072
1073
1074
1075
1076
1077
1078
1079
1080
1081
1082
1083
1084
1085
1086
1087
1088
1089
1090
1091
1092
1093
1094
1095
1096
1097
1098
1099
1100
1101
1102
1103
1104
1105
1106
1107
1108
1109
1110
1111
1112
1113
1114
1115
1116
1117
1118
1119
1120
112

In [12]:
#  prompt = "Joan found 70 seashells on the beach . she gave Sam some of her seashells . She has 27 seashell . How many seashells did she give to Sam ?"


#     # prompt_template=f'''[INST] <<SYS>>
#     # You are an online math tutor bot. Follow the given instructions: Given the below-structured query,
#     # your job is to understand the word problem mentioned as "question" and explain the solution in numbered
#     # steps to get the correct "answer". Begin your response with "let's think step by step" and append Answer: ["answer" from the
#     # below-structured query] at the end of your reasoning. Strictly limit the reasoning to 20-30 words. Do not use emojis.{prompt}[/INST]
#     # '''
# prompt_template=f'''[INST] <<SYS>>Lets think step by step. {prompt}[/INST]
#     '''
# print("*** Pipeline:")
# pipe = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     max_new_tokens=512,
#     do_sample=True,
#     temperature=0.1,
#     top_p=0.95,
#     top_k=1,
#     repetition_penalty=1.1
# )

# print(pipe(prompt_template)[0]['generated_text'])

In [13]:
# print(results)

In [14]:
import json

In [15]:
# with open('/content/drive/MyDrive/datasets/commonsense_training.json', 'w') as file:
#      json.dump(results, file, indent=4)

# Existing JSON file path
file_path = '/content/drive/MyDrive/datasets/commonsense_training.json'

# Load existing JSON content from the file
existing_data = []
try:
    with open(file_path, 'r') as file:
        existing_data = json.load(file)
except FileNotFoundError:
    # Handle the case where the file doesn't exist yet (i.e., first time appending)
    pass

# Append new results to the existing data
existing_data.extend(results)

# Write the updated data (existing + new) back to the file
with open(file_path, 'w') as file:
    json.dump(existing_data, file, indent=4)

In [16]:
# # prompt = "Joan found 70 seashells on the beach . she gave Sam some of her seashells . She has 27 seashell . How many seashells did she give to Sam ?"
# prompt = '''{
#             "sample_index": 1,
#             "question": "There were 28 bales of hay in the barn . Tim stacked bales in the barn today . There are now 54 bales of hay in the barn . How many bales did he store in the barn ?",
#             "answer": "26"
#         }'''

# prompt_template=f'''[INST] <<SYS>>
# You are an online math tutor bot. Follow the given instructions: Given the below-structured query,
# your job is to understand the word problem mentioned as "question" and explain the solution in numbered
# steps to get the correct "answer". Begin your response with "let's think step by step" and append Answer: ["answer" from the
# below-structured query] at the end of your reasoning. Strictly limit the reasoning to 20-30 words. Do not use emojis.{prompt}[/INST]
# '''

# print("*** Pipeline:")
# pipe = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     max_new_tokens=512,
#     do_sample=True,
#     temperature=0.1,
#     top_p=0.95,
#     top_k=5,
#     repetition_penalty=1.1
# )

# print(pipe(prompt_template)[0]['generated_text'])